## Introduction


Please **Run this notebook to explore crater detection model**

### initialisation

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os
import re
# import torch
from yolov5 import utils
import torch
import torchvision
from PIL import Image
import ipyplot
from IPython.display import display
from pathlib import Path
# yolov5
from yolov5 import crater_tools

### set the workspace path

In [6]:
# the workspace path, your current file path
# os.getcwd()
# # os.chdir('/Users/yd1522/ACSE-2/group_project')
workspace_path = os.getcwd()
workspace_path

'/Users/yd1522/ACSE-2/group_project'

## Model input
please input the test path 

**input the test_path**

In [7]:
inputbounded_test_path = widgets.Text(
    description='test_path: ',
    value = '/workspace/dataset/test'
)

**input the CDM output path**

In [8]:
inputbounded_output_path = widgets.Text(
    description='output_path: ',
    value = '/workspace/CDM/moon'
)

**input the location of the image centre**

In [10]:
inputbounded_image_centre = widgets.BoundedFloatText(
    description='centre: ',
    value = 0
)

**input the image width**

In [9]:
inputbounded_image_width = widgets.BoundedFloatText(
    description='width: ',
    value = 0
)

**input the image height**

In [11]:
inputbounded_image_height = widgets.BoundedFloatText(
    description='height: ',
    value = 0
)

**input the image resolution**

In [12]:
inputbounded_image_resolution = widgets.BoundedFloatText(
    description='resolution: ',
    value = 0
)

**input the radius of the target planet**

In [13]:
inputbounded_planet_radius = widgets.BoundedFloatText(
    description='radius: ',
    value = 0
)

**built the input GUI framework** to double check

In [14]:
output_1 = widgets.Output()
input_1_box = widgets.HBox([inputbounded_test_path, inputbounded_output_path])

In [15]:
input_2_box = widgets.HBox([inputbounded_image_centre, inputbounded_image_width, inputbounded_image_height])
input_box = widgets.VBox([input_1_box, input_2_box])

In [16]:
input_3_box = widgets.HBox([inputbounded_image_resolution, inputbounded_planet_radius])
input_box = widgets.VBox([input_1_box, input_2_box,input_3_box ])
input_widgets1 = widgets.VBox([input_box])
btn_input = widgets.Button(description='Input')

tab = widgets.Tab([output_1])

In [17]:
input_widgets_ = widgets.HBox([btn_input])
input_widgets3 = widgets.VBox([input_widgets1,input_widgets_])
dashboard1 = widgets.VBox([input_widgets3, tab])


In [18]:
def user_input():
    
    print("Your test dataset path is: {}".format(inputbounded_test_path.value))
    print("Your CDM output path is: {}".format(inputbounded_output_path.value))
    print("Your image centre in latitude and longitude is: {}".format(inputbounded_image_centre.value))
    print("Your image width in dgrees is: {}".format(inputbounded_image_width.value))
    print("Your image height is dgrees is: {}".format(inputbounded_image_height.value))
    print("Your image resolution  is: {} (m/px)".format(inputbounded_image_resolution.value))
    print("Your input radius of the target planet is: {}".format(inputbounded_planet_radius.value))



In [120]:
def get_result():
    output_1.clear_output()
    with output_1:
        df_result = user_input()
        
def btn_input_eventhandler(obj):
    get_result()

btn_input.on_click(btn_input_eventhandler)

In [23]:
# test_path /Users/yd1522/ACSE-2/example-data/mars
# output_path /Users/yd1522/ACSE-2/CDM

**input here :D**
- **click** the button **[input]** for double check your input parameters
- for centre, width, height, resolution, radius is set to be 0 as default, but they are not used in our model currently
    - for future improvement, we will create python script and use above parameters(c, w, h etc.) as api to do dataset preprocessing

In [25]:
display(dashboard1)

## Mars CDM

### user-guide & workflow

- **Step1**: use crater tool **generate_test_files.py** to automatically 
    - get dataset from your input path
    - generate new $.yaml$ file for validate in test dataset


- **Step2**: run **detect.py** to get predictions 
    - for directly using the interface, please set the API **--exist_ok** set **True**
    - use crater tool "plot_two_boxes.py" to get images with


- **Step3**: run **val.py** with crater CDM API (**--val-test**)
    - for directly using the interface, please set the API **--exist_ok** set **True**
    - get the statistics analysis results over the whole test dataset
    - name: val-test-statistics-results.csv
    
    
- **Step4**: use crater tool **get_bbox_csv.py** to get get detections csv files

    
- **Step5**: run crater tool **generate_output_results.py** to automatically get results in output path

### Mars Task GUI

In [47]:
Mars_Task = widgets.Dropdown(
    options = ['Start', 'Step1', 'Step2', 'Step3', 'Step5'],
    value = 'Start',
    description = 'Mars Task: '
)
# Mars_Task

In [48]:
Mars_Task_output_gui = widgets.Output()
Mars_Task_tab = widgets.Tab([Mars_Task_output_gui])
# Mars_Task_tab

In [49]:
Mars_Task_dashboard_gui = widgets.VBox([Mars_Task, Mars_Task_tab])
# Mars_Task_dashboard_gui

In [62]:
def Mars_Task_gui_func(change):
    with Mars_Task_output_gui:
        Mars_Task_output_gui.clear_output()
        if change['new'] == 'Step1':

            os.chdir(workspace_path+'/yolov5/crater_tools')
            !python generate_test_files.py $inputbounded_test_path.value $'mars'

        elif change['new'] == 'Step2':
            # detect
            os.chdir(workspace_path+'/yolov5')
            !python detect.py --weights /weights/best_mars_1.pt --conf 0.25 --source ../yolov5/datasets/mars/images/mytest --exist-ok
            ## plot_two_bboxes
            os.chdir(workspace_path+'/yolov5/crater_tools')
            !python plot_two_boxes.py
            
            
        elif change['new'] == 'Step3':
            # Validate your chosen weights on your test datset
            os.chdir(workspace_path+'/yolov5')
            !python val.py --weights /weights/best_mars_1.pt --data ../yolov5/data/mars-detect.yaml --exist-ok --val-test

        elif change['new'] == 'Step4':
            # get detections
            os.chdir(workspace_path+'/yolov5/crater_tools')
            !python get_bbox_csv.py $'mars' 
            
        elif change['new'] == 'Step5':
            # get output results
            os.chdir(workspace_path+'/yolov5/crater_tools')
            !python generate_output_results.py $'mars' 
            
        else:
            # :D
            print("\n")
            print("**********************Please Select a Mars Task :D ***************")


In [63]:
Mars_Task.observe(Mars_Task_gui_func, names='value')

**Mars Task GUI**

In [66]:
display(Mars_Task_dashboard_gui)

###  load Mars CDM results

In [74]:
import re
image_path = '/Users/yd1522/ACSE-2/group_project//Mars_THEMIS_Training/images'
os.chdir(workspace_path)
path = os.getcwd()
rel_path = os.path.relpath(image_path, path)
files = os.listdir(rel_path)
images = [rel_path + '/' +
          files[i] for i in range(len(files)) 
          if re.search(".pn", files[i]) is not None]

#get all categories
kinds = [image.split('_')[-3] for image in images]
kinds_ = [kind.split('/')[-1] for kind in kinds]

**load Mars detect data**

In [76]:
# /Users/yd1522/ACSE-2/group_project/yolov5/runs/mars
image_path_de = '/Users/yd1522/ACSE-2/group_project//yolov5/runs/mars/detect/exp'
path_de = os.getcwd()
rel_path_de = os.path.relpath(image_path_de, path_de)
files_de = os.listdir(rel_path_de)
images_de = [rel_path_de + '/' +
          files_de[i] for i in range(len(files_de)) 
          if re.search(".pn", files_de[i]) is not None]

#get all categories
kinds_de = [image.split('_')[-3] for image in images_de]
kinds_de_ = [kind.split('/')[-1] for kind in kinds_de]
# kinds_de_

**load Mars detect comparsion**

In [77]:
# /Users/yd1522/ACSE-2/group_project/yolov5/runs/mars
image_path_comp = '/Users/yd1522/ACSE-2/group_project//yolov5/runs/plot/exp9'
path_comp = os.getcwd()
rel_path_comp = os.path.relpath(image_path_comp, path_comp)
files_comp = os.listdir(rel_path_comp)
images_comp = [rel_path_comp + '/' +
          files_comp[i] for i in range(len(files_comp)) 
          if re.search(".pn", files_comp[i]) is not None]

#get all categories
kinds_comp = [image.split('_')[-3] for image in images_comp]
kinds_comp_ = [kind.split('/')[-1] for kind in kinds_comp]
# kinds_comp_

**load Mars S-F plot**

### Mars CDM Gui framework

In [78]:
plot_1_Mars = widgets.Output()
plot_2_Mars = widgets.Output()

plot_3_Mars = widgets.Output()
plot_4_Mars = widgets.Output()
analysis_mars = widgets.Output()

btn_analysis_mars = widgets.Button(description='Analysis')

In [79]:
def common_analysis_mars():
    plot_1_Mars.clear_output()
    plot_2_Mars.clear_output()
    plot_3_Mars.clear_output()
    plot_4_Mars.clear_output()
    analysis_mars.clear_output()

    
    with plot_1_Mars:
        
        ipyplot.plot_class_tabs(images, kinds_, max_imgs_per_tab=10, img_width=150)
        
        
    with plot_2_Mars:

        ipyplot.plot_class_tabs(images_de, kinds_de_, max_imgs_per_tab=10, img_width=150)
        
        
    with plot_3_Mars: 

        ipyplot.plot_class_tabs(images_comp, kinds_comp_, max_imgs_per_tab=10, img_width=150)

    
    with plot_4_Mars:
        !python example.py
    
    with analysis_mars: # M-z Plot
        df_result = pd.read_csv("/Users/yd1522/ACSE-2/group_project/val-test-statistics-results.csv", index_col=0)
        display(df_result)

    
def btn_analysis_eventhandler_mars(obj):
    common_analysis_mars()

btn_analysis_mars.on_click(btn_analysis_eventhandler_mars)


In [80]:
input_widgets2 = widgets.HBox([btn_analysis_mars])

In [83]:
# tab = widgets.Tab([output, output_2, plot_1, plot_2, plot_3, plot_4, plot_5, plot_6])
tab_mars = widgets.Tab([plot_1_Mars, plot_2_Mars, plot_3_Mars, plot_4_Mars, analysis_mars])

tab_mars.set_title(0, 'Original Image')
tab_mars.set_title(1, 'CDM Detection')
tab_mars.set_title(2, 'Results Comparison')
tab_mars.set_title(3, 'Size-Frequency Plot') # size-frequency distribution of detected craters
tab_mars.set_title(4, 'Statistics Analysis')
# display(tab_mars)

In [85]:
dashboard2 = widgets.VBox([input_widgets2, tab_mars])
# display(dashboard2)

## Moon CDM

### user-guide & workflow

- **Step1**: use crater tool **Preproc.py** to automatically 
    - split the moon dataset images
    - from more details see the notebook in **Moon-preproc.ipynb**
    
    
- **Step2**: use crater tool **generate_test_files.py** to automatically 
    - get dataset from your input path
    - generate new $.yaml$ file for validate in test dataset


- **Step3**: run **detect.py** to get predictions 
    - for directly using the interface, please set the API **--exist_ok** set **True**


- **Step4**: run **val.py** with crater CDM API (**--val-test**)
    - for directly using the interface, please set the API **--exist_ok** set **True**
    - get the statistics analysis results over the whole test dataset
    - name: val-test-statistics-results.csv
    
    
- **Step5**: run crater tool **generate_output_results.py** to automatically get results in output path
    - for more details, see the notebook named **Visualisation.ipynb**

### Moon Task GUI

In [121]:
Moon_Task = widgets.Dropdown(
    options = ['Start', 'Step1', 'Step2', 'Step3', 'Step5'],
    value = 'Start',
    description = 'Moon Task: '
)
# Mars_Task

In [122]:
Moon_Task_output_gui = widgets.Output()
Moon_Task_tab = widgets.Tab([Moon_Task_output_gui])
Moon_Task_dashboard_gui = widgets.VBox([Moon_Task, Moon_Task_tab])

In [123]:
def Moon_Task_gui_func(change):
    with Moon_Task_output_gui:
        Moon_Task_output_gui.clear_output()
        
        if change['new'] == 'Step1':
            # split the moon dataset
            # !python Preproc.py for example

        elif change['new'] == 'Step2':
            # test files generate
            os.chdir(workspace_path+'/yolov5/crater_tools')
            FILE = Path(inputbounded_test_path.value).resolve()
            new_test_path = str(FILE.parents[0]) + '/moon-temp'
            !python generate_test_files.py $new_test_path $'moon'

        elif change['new'] == 'Step3':
            # detect
            os.chdir(workspace_path+'/yolov5')
            !python detect.py --weights /weights/best_moon_1.pt --conf 0.25 --source ../yolov5/datasets/moon/images/mytest --exist-ok

        elif change['new'] == 'Step4':
            # Validate your chosen weights on your test datset
            os.chdir(workspace_path+'/yolov5')
            !python val.py --weights /weights/best_moon_1.pt --data ../yolov5/data/moon-detect.yaml --exist-ok --val-test
            
            
        elif change['new'] == 'Step5':
            # get output results
            os.chdir(workspace_path+'/yolov5/crater_tools')
            !python generate_output_results.py $'moon' 

        else:
            # :D
            print("\n")
            print("**********************Please Select a Task :D ***************")


**Moon Task GUI**

In [125]:
display(Moon_Task_dashboard_gui)

###  load Moon CDM results

**load Moon orginal image**

In [118]:
import re
image_path_m = '/Users/yd1522/ACSE-2/group_project//Moon_WAC_Training/images'
os.chdir(workspace_path)
path_m = os.getcwd()
rel_path_m = os.path.relpath(image_path_m, path_m)
files_m = os.listdir(rel_path_m)
images_m = [rel_path_m + '/' +
          files_m[i] for i in range(len(files_m)) 
          if re.search(".jp", files_m[i]) is not None]
# images_m

In [119]:
kinds_m = [image.split('_')[-1] for image in images_m]
kinds_m_ = [kind.split('.')[0] for kind in kinds_m]
# kinds_m_

**load Moon detect image**

**load Moon Size-Frequency**

##  Moon CDM Gui framework

In [98]:
plot_1_Moon = widgets.Output()
plot_2_Moon = widgets.Output()

plot_3_Moon = widgets.Output()
plot_4_Moon = widgets.Output()
analysis_Moon = widgets.Output()

btn_analysis_M = widgets.Button(description='Analysis')

In [99]:
def common_analysis_M():
    plot_1_Moon.clear_output()
    plot_2_Moon.clear_output()
    plot_3_Moon.clear_output()
    plot_4_Moon.clear_output()
    analysis_Moon.clear_output()

    
    with plot_1_Moon:
        
        ipyplot.plot_class_tabs(images_m, kinds_m_, max_imgs_per_tab=10, img_width=800)
        
        
    with plot_2_Moon:

        !python example.py
        
    with plot_3_Moon: 

        !python example.py

    
    with plot_4_Moon: # Z-dt Plot
        !python example.py
    
    with analysis_Moon: # M-z Plot
        !python example.py

    
def btn_analysis_eventhandler_M(obj):
    common_analysis_M()

btn_analysis_M.on_click(btn_analysis_eventhandler_M)


In [100]:
input_widgets2_M = widgets.HBox([btn_analysis_M])

In [103]:
# tab = widgets.Tab([output, output_2, plot_1, plot_2, plot_3, plot_4, plot_5, plot_6])
tab_M = widgets.Tab([plot_1_Moon, plot_2_Moon, plot_3_Moon, plot_4_Moon, analysis_Moon])

tab_M.set_title(0, 'Original Image')
tab_M.set_title(1, 'CDM Detection')
tab_M.set_title(2, 'large Image')
tab_M.set_title(3, 'S-F Plot') # size-frequency distribution of detected craters
tab_M.set_title(4, 'Statistics Analysis')
# display(tab)

In [133]:
dashboard2_M = widgets.VBox([input_widgets2_M, tab_M])
# display(dashboard2_M)

## Results Visualisation

**GUI framework**

In [105]:
CDM_Planet = widgets.RadioButtons(
    options=['Mars','Moon'],
    value='Mars',
    description='Planet: '
)

In [108]:
output_gui = widgets.Output()
tab_gui = widgets.Tab([output_gui])
# tab_gui

In [110]:
dashboard_gui = widgets.VBox([CDM_Planet, tab_gui])
# dashboard_gui                          

In [115]:
def myfunc_gui(change):
    with output_gui:
        output_gui.clear_output()
        if change['new'] == 'Mars':
            display(dashboard2)
        elif change['new'] == 'Moon':
            display(dashboard2_M)
        else:
            print('*********************** please select the button :D ***********************')
    

In [116]:
CDM_Planet.observe(myfunc_gui, names='value')

In [117]:
display(dashboard_gui)